In [ ]:
import pandas as pd

In [ ]:
dados = pd.read_json('https://github.com/lukaswyllis/formacao_python_para_data_science/raw/refs/heads/master/dados-curso7/dados_hospedagem.json')
dados.head()

In [ ]:
dados = pd.json_normalize(dados['info_moveis'])
dados

In [ ]:
colunas = list(dados.columns)
colunas

In [ ]:
dados = dados.explode(colunas[3:])
dados

In [ ]:
dados.reset_index(inplace = True,drop=True)
dados.head()

In [ ]:
dados.info()

In [ ]:
import numpy as np

In [ ]:
dados['max_hospedes']=dados['max_hospedes'].astype(np.int64)

In [ ]:
dados.info()

In [ ]:
col_numericas = ['quantidade_banheiros','quantidade_quartos','quantidade_camas']

In [ ]:
dados[col_numericas] = dados[col_numericas].astype(np.int64)

In [ ]:
dados['avaliacao_geral'] = dados['avaliacao_geral'].astype(np.float64)

In [ ]:
dados.info()

In [ ]:
dados['preco'] = dados['preco'].apply(lambda x: x.replace('$','').replace(',','').strip())

In [ ]:
dados['preco'] = dados['preco'].astype(np.float64)

In [ ]:
dados.info()

In [ ]:
dados[['taxa_deposito','taxa_limpeza']] = dados[['taxa_deposito','taxa_limpeza']].applymap(lambda x: x.replace('$','').replace(',','').strip())

In [ ]:
dados[['taxa_deposito','taxa_limpeza']] = dados[['taxa_deposito','taxa_limpeza']].astype(np.float64)

In [ ]:
dados.info()

In [ ]:
dados['descricao_local'] = dados['descricao_local'].str.lower()

In [ ]:
dados.head()

In [ ]:
dados['descricao_local'][3169]

In [ ]:
dados['descricao_local'] = dados['descricao_local'].str.replace('[^a-zA-Z0-9\-\']',' ',regex=True)

In [ ]:
dados['descricao_local'] = dados['descricao_local'].str.replace('(?<!\w)-(?!\w)',' ',regex=True)

In [ ]:
dados['descricao_local'] = dados['descricao_local'].str.split()
dados.head()

In [ ]:
dados['comodidades'] = dados['comodidades'].str.replace('\{|}|\"','',regex=True)

In [ ]:
dados['comodidades'] = dados['comodidades'].str.split(',')
dados.head()

In [ ]:
dt_data = pd.read_json('https://github.com/lukaswyllis/formacao_python_para_data_science/raw/refs/heads/master/dados-curso7/moveis_disponiveis.json')
dt_data.head()

In [ ]:
dt_data.info()

In [ ]:
dt_data['data'] = pd.to_datetime(dt_data['data'])

In [ ]:
dt_data.info()

In [ ]:
dt_data.head()

In [ ]:
dt_data['data'].dt.strftime('%Y-%m')

In [ ]:
subset = dt_data.groupby(dt_data['data'].dt.strftime('%Y-%m'))['vaga_disponivel'].sum()
subset